# WCT Analysis

Whistle Contour Types (WCT) analysis. 

Exploration of the categories produced by ARTwarp and manual verifications

## Importations

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.patches import Patch

from WCT_analysis_utils import (add_file_sequence_cols, run_SIGID, plot_wct_grid,
    plot_hue_wct_grid, categories_multi_dates, kw_test, pairwise_tests, get_stars)

## Import data

In [ ]:
results_df = pd.read_csv(
    "./resources/Verification_outputs/final_categories_1s_96percent_smooth.csv",
    index_col=0, parse_dates=["start_dt", "stop_dt"])   
results_df["date"] = pd.to_datetime(results_df["start_dt"].dt.date)
results_df["year"] = results_df["date"].dt.year
results_df["clean_activation_state"] = results_df["activation_state"].copy()
results_df.loc[results_df.clean_activation_state == 'Control', 'clean_activation_state'] = "BEF"
results_df = add_file_sequence_cols(results_df)

## SIGID

A method for the IDentification of SIGnature whistles (SIGID) developped by [Janik et al. 2013](https://onlinelibrary.wiley.com/doi/10.1111/j.1748-7692.2011.00549.x).

It relies on the analysis of WCT: if 75% or more of the whistles of a WCT have another representant of their types within a time window of 1-10s around them, then this WCT can be considered to be a SWT (Signature Whistles Type).

In [Fearey et al. 2019](http://link.springer.com/10.1007/s10071-019-01274-1), the authors consider a time-window of 0.2-10s for their study on common dolphins. As we are also working with *D.delphis*, we will test this interval as well.

### SWT identification

Identification of SWTs among the WCTs

In [ ]:
intervals_to_try = [(0.2, 10), (1, 10)] # min and max for each time-window

In [ ]:
# Identify SWTs (SIGID method)
for interval in tqdm(intervals_to_try, desc="SIGID interval", leave=True, position=0):
    results_df = run_SIGID(results_df, interval=interval)

# Difference in the number of SWT identified
results_df["SWT_difference"] = (
    results_df[f"SWT_[{intervals_to_try[0][0]},{intervals_to_try[0][1]}]"] !=
    results_df[f"SWT_[{intervals_to_try[1][0]},{intervals_to_try[1][1]}]"]
)
mismatch_whistles = results_df['SWT_difference'].sum()
mismatch_category = ((results_df.groupby('category')["SWT_difference"].sum()) > 1).sum()

print(f"\nDifferent intervals: {mismatch_category} mismatched SWTs (or {mismatch_whistles} SWs)")


### Choosing SIGID interval

So using the original interval of [1, 10] sec we get different results than if we use the [0.2, 10] sec interval.

Which categories are classified differently?

In [ ]:
mismatch_df = results_df[results_df["SWT_difference"]].copy()
for id_row, row in mismatch_df.iterrows():
    if row["SWT_[1,10]"]:
        mismatch_df.loc[id_row, "category"] = str(row["category"])+"_[1,10]"
    else:
        mismatch_df.loc[id_row, "category"] = str(row["category"])+"_[0.2,10]"

misfig, misaxs = plot_wct_grid(mismatch_df, name="SWT ", rename=False)
misfig.suptitle("SWTs mismatch when applying different intervals to SIGID")
misfig.text(
    0.5, 0.925,
    "SWT categories are accompanied with the interval used to identify them.",
    ha="center", fontsize=9)
misfig.subplots_adjust(bottom=0.085, top=0.875)
misfig.set_size_inches(10, 6)
misfig.savefig("/home/loic/Pictures/CDD/SWTs_interval_mismatch.pdf")
plt.show()

Ok, so all the additional SWTs come from the selection of WCTs using the [0.2, 10] sec interval.

Since, this interval was specifically create for *D. delphis* whistles, we keep this interval for further analyses.

In [ ]:
the_interval = "SWT_[0.2,10]"
print(f"Choosing interval: {the_interval}")
results_df['is_SW'] = results_df[the_interval].astype(int)

## SWT data summary

In [ ]:
print("\nAmong all whistles:")
print(f"\tSWTs represent {100*results_df[results_df[the_interval]].category.nunique()/results_df.category.nunique():.2f}% of WCTs ({results_df[results_df[the_interval]].category.nunique()}/{results_df.category.nunique()}).")
print(f"\tSWs represent {100*results_df[the_interval].sum()/len(results_df):.2f}% of whistles ({results_df[the_interval].sum()}/{len(results_df)}).")

results_df_several = results_df[results_df.groupby('category')['category'].transform('count') > 1].copy()
print("Among categories with more than 1 whistle:")
print(f"\tSWTs represent {100*results_df_several[results_df_several[the_interval]].category.nunique()/results_df_several.category.nunique():.2f}% of WCTs ({results_df_several[results_df_several[the_interval]].category.nunique()}/{results_df_several.category.nunique()}).")
print(f"\tSWs represent {100*results_df_several[the_interval].sum()/len(results_df_several):.2f}% of whistles ({results_df_several[the_interval].sum()}/{len(results_df_several)}).\n")

In [ ]:
# Calculate category sizes and their frequencies
category_sizes = results_df['category'].value_counts()
size_distribution = pd.DataFrame({
    'Category Size': category_sizes.values,
    'Number of Categories': [1] * len(category_sizes)
}).groupby('Category Size')['Number of Categories'].sum().reset_index()

# Create the bar plot
sns.barplot(
    data=size_distribution, 
    x='Category Size', y='Number of Categories',
    color="gray", edgecolor="black")

plt.title('Distribution of WCTs sizes')
plt.xlabel('Category size (number of whistles per category)')
plt.ylabel('Number of Categories')

# Adjust layout to prevent label cutoff
plt.tight_layout()
plt.yscale('log')
plt.savefig("/home/loic/Pictures/CDD/WCTs_distribution.pdf")
plt.show()

In [ ]:
# Calculate category sizes and their frequencies
category_sizes = results_df[results_df["is_SW"]==1]['category'].value_counts()
size_distribution = pd.DataFrame({
    'Category Size': category_sizes.values,
    'Number of Categories': [1] * len(category_sizes)
}).groupby('Category Size')['Number of Categories'].sum().reset_index()

# Create the bar plot
sns.barplot(
    data=size_distribution, 
    x='Category Size', y='Number of Categories',
    color="gray", edgecolor="black")

plt.title('Distribution of SWTs sizes')
plt.xlabel('Category size (number of whistles per category)')
plt.ylabel('Number of Categories')

# Adjust layout to prevent label cutoff
plt.tight_layout()
plt.yscale('log')
plt.savefig("/home/loic/Pictures/CDD/SWTs_distribution.pdf")
plt.show()

In [ ]:
# is each WCT or SWT specific to a specific sequence (aka group of dolphin?)
print(f"Average number of sequences associated to a specific WCT: {results_df[results_df['is_SW']==0].groupby('category_remap')['sequence'].nunique().mean()}")
print(f"Average number of sequences associated to a specific SWT: {results_df[results_df['is_SW']==1].groupby('category_remap')['sequence'].nunique().mean()}")

# very slight difference but overall, one WCT = 1 group of dolphin.

In [ ]:
# display best contours
gridfig, gridaxs = plot_wct_grid(
    results_df_several, mode="median_dur",
    name="SWT", n_categories=25)
gridfig.suptitle("Top-25 Signature Whislte Types (SWTs) with the most elements")
gridfig.text(
    0.5, 0.925,
    "Whislte contours are represented in gray, black lines are contours of median duration for visualisation purposes",
    ha="center", fontsize=9)
gridfig.subplots_adjust(bottom=0.085, top=0.875)
gridfig.set_size_inches(10, 6)
for axs in gridaxs:
    for ax in axs:
        ax.get_children()[-11].set_height(5)
        ax.set_title(
            ax.get_children()[-4].get_text(),
            pad=2.5, 
            fontsize=9,
            fontweight='bold')
gridfig.savefig("/home/loic/Pictures/CDD/SWTs_25.pdf")
plt.show()

In [ ]:
# display uncategorised contours
sinfig, sinaxs = plot_wct_grid(
    results_df[results_df.groupby('category')['category'].transform('count') == 1].copy(), 
    name="WCT", n_categories=25)
sinfig.suptitle("25 Randomly selected Whistle Contour Types (WCTs)")
sinfig.text(
    0.5, 0.925,
    "These contours are unique, each one appears only once in our dataset",
    ha="center", fontsize=9)
sinfig.subplots_adjust(
    bottom=0.085, top=0.875, 
    hspace=0.42, wspace=0.066)
sinfig.set_size_inches(10, 6)
for axs in sinaxs:
    for ax in axs:
        ax.get_children()[-11].set_height(5)
        ax.set_title(
            ax.get_children()[-4].get_text(),
            pad=3, 
            fontsize=9,
            fontweight='bold')
sinfig.savefig("/home/loic/Pictures/CDD/WCTs_25.pdf")
plt.show()

## SWT data exploration

Let's investigate if any of the variables that we measured has an effect on the number/proportion of SWTs. Starting with dates.

### Shared SWTs

In [ ]:
print(
    categories_multi_dates(results_df[results_df[the_interval]], "year").category.nunique(),
    f"SWTs are shared between years.")
print(
    categories_multi_dates(results_df[results_df[the_interval]], "date").category.nunique(),
    f"SWTs are shared between days.")
print(
    categories_multi_dates(results_df[results_df[the_interval]], "sequence").category.nunique(),
    f"SWTs are shared between sequences.")

Ok so we have only 2 SWTs that are shared between 2 sequences, let's visualise them.

In [ ]:
shared_sequence = categories_multi_dates(results_df[results_df[the_interval]], "sequence")
# plot them per sequence
seqfig, axseq = plot_hue_wct_grid(shared_sequence, hue="sequence", name="SWT", legend_title="Sequence start")
seqfig.suptitle("Common SWTs found in 2 distinct sequences.")
seqfig.subplots_adjust(
    bottom=0.2, top=0.82,
    left=0.13)
seqfig.set_size_inches(8, 2.66)
for axs in axseq:
    for ax in axs:
        ax.set_title(
            ax.get_children()[-5].get_text(),
            pad=2.1,
            fontsize=8,
            fontweight='bold')
seqfig.savefig("/home/loic/Pictures/CDD/SWTs_shared_sequences.pdf")
plt.show()

These SWTs shared between 2 sequences could have multiple explanations:
- Different groups contained the same 2 individuals, that were not noticed by the observers onboard.
- Contours with very similar shapes, but with fine variations, associated to different but closely related individuals.
- misclassification errors of the ARTwarp algorithm (fine variations are visible, but would need a stricter vigilance value to be differentiated).

Both these hypotheses are plausible. In practice, shared SWTs are very rare in our dataset.

In [ ]:
print(
    categories_multi_dates(results_df, "year").category.nunique(),
    f"WCTs are shared between years.")
print(
    categories_multi_dates(results_df, "date").category.nunique(),
    f"WCTs are shared between days.")
print(
    categories_multi_dates(results_df, "sequence").category.nunique(),
    f"WCTs are shared between sequences.")

When considering all WCTs, more types are shared between sequences/days. But by definition, these WCTs are not specific to individuals as they are not SWs, so they could be shared by several dolphins and/or groups of dolphins, with various applications that are unknown to this day.

### SWTs proportion

Let's investigate if the proportion SWs/whistles varies depeding on our recorded variables.

Why? Because I've seen it here [Janik et al. 1994](http://link.springer.com/10.1007/BF00170704)

#### Years

In [ ]:
print("Fisher's exact tests:")
test_years = pairwise_tests(results_df, "year", "is_SW")
for test_year in test_years:
    print("\t", test_year[0], "VS", test_year[1], ":", f"{test_year[2]:.2e}", get_stars(test_year[2]))

In [ ]:
# prepare plot
letters = ["a", "b", "a"] # manually from previous cell

sns.set_style("ticks")
fig_year, axs_year = plt.subplots(1,1)
sns.barplot(
    data=results_df, x="year", y="is_SW",
    capsize=0.05, width=0.8, 
    n_boot=10000, seed=42, ax=axs_year,
    color="gray", edgecolor="black"
)
axs_year.set_ylim(0,0.215)
for i, x in enumerate(list(results_df.year.unique())):
    axs_year.text(
        str(x), 0.211, letters[i], ha="center", va="top", fontsize=11,
        backgroundcolor="white"
    )
fig_year.suptitle("Proportion of SWs among whistles per year")
fig_year.text(
    0.5, 0.89,
    "Different letter show significant differences\naccording to Fisher's exact tests (p-values < 0.05).\nError bars show 95% confidence intervals.",
    ha="center", va="bottom", fontsize=8)
axs_year.set_xlabel("Year")
axs_year.set_ylabel("SWs (in % of whistles)")
fig_year.set_size_inches(3, 6)
fig_year.subplots_adjust(
    top=0.88
)
axs_year.xaxis.grid(True)
axs_year.yaxis.grid(True)
fig_year.savefig("/home/loic/Pictures/CDD/barplot_years.pdf")
plt.show()
sns.reset_orig()

Some notable differences, between 2021 and the other years. But 2021 was a year with only very few observations, so these differences are expected.

#### Days

In [ ]:
print("Fisher's exact tests: (uncomment to show)")
test_days = pairwise_tests(results_df, "date", "is_SW")
# for test_day in test_days:
#     print("\t", test_day[0], "VS", test_day[1], ":", f"{test_day[2]:.2e}", get_stars(test_day[2]))

In [ ]:
letters = [
    "a", "b", "a", "a", "c", "a", "a", "c", "b", "a", 
    "abc", # modified from the tests because only 3 data points, so test is not reliable
    "NA", "NA"] # manually from previous cell

sns.set_style("ticks")
fig_day, axs_day = plt.subplots(1,1)
sns.barplot(
    data=results_df, x="date", y="is_SW",
    capsize=0.05, width=0.8, 
    n_boot=10000, seed=42, ax=axs_day,
    color="gray", edgecolor="black"
)

# plot with statistics
axs_day.set_ylim(0,1.1)
for i, x in enumerate(list(results_df.date.dt.date.unique())):
    axs_day.text(
        str(x), 1.075, letters[i], ha="center", va="top", fontsize=10,
        backgroundcolor="white"
    )
axs_day.set_xlabel("Day of recording")
axs_day.set_ylabel("SWs (in % of whistles)")
for label in axs_day.get_xticklabels():
    label.set_rotation(45)
    label.set_ha('right')
fig_day.suptitle("Proportion of SWs among whistles per recording day")
fig_day.text(
    0.5, 0.89,
    "Different letter show significant differences according to Fisher's exact tests (p-values < 0.05).\nError bars show 95% confidence intervals.",
    ha="center", va="bottom", fontsize=8)
fig_day.subplots_adjust(
    top=0.875
)
axs_day.xaxis.grid(True)
axs_day.yaxis.grid(True)
fig_day.savefig("/home/loic/Pictures/CDD/barplot_days.pdf")
sns.reset_orig()
plt.show()

In [ ]:
a_bars = results_df[(
    (results_df.date.dt.date.astype(str) == "2020-07-11") |
    (results_df.date.dt.date.astype(str) == "2020-07-13") |
    (results_df.date.dt.date.astype(str) == "2020-07-16") |
    (results_df.date.dt.date.astype(str) == "2020-07-18") |
    (results_df.date.dt.date.astype(str) == "2021-07-09") |
    (results_df.date.dt.date.astype(str) == "2022-07-18")
)]
print("Proportion of SWs in 'a' distributions:", f"{100*a_bars[the_interval].sum()/len(a_bars):.2f}%")
print(f"2022-07-20: {len(results_df[results_df.date.dt.date.astype(str) == '2022-07-20'])} whistles this day.")

Some noticeable features : 
- Most days, rates of SWs are low (5% of all whistles for 'a' distributions)
- 2022-07-20: an outlier, only 3 whistles this day, of which 2 are SWs.

For days 2020-07-12, 2020-07-17, 2022-07-16 and 2022-07-17 that are similar outliers, there might be some explanation but I cannot find it for now, let's explore the other variables first before.

All in all, rates are usually low, but great variations can appear, let's see if its not due to other features, such as behaviour, group size or our beacon.

#### Behavioural state

In [ ]:
# test data
print("Fisher's exact tests:")
test_behaviours = pairwise_tests(results_df, "behaviour", "is_SW")
for test_behaviour in test_behaviours:
    if type(test_behaviour[0])==str and type(test_behaviour[1])==str:
        print("\t", test_behaviour[0], "VS", test_behaviour[1], ":", f"{test_behaviour[2]:.2e}", get_stars(test_behaviour[2]))

In [ ]:
# Make plot
letters = ["a", "ab", "b", "b", "b"] # manually from previous cell
behavioural_states = ["Milling", "Socialising", "Travelling", "Foraging", "Attraction"]

sns.set_style("ticks")
fig_behaviour, axs_behaviour = plt.subplots(1,1)
sns.barplot(
    data=results_df, x="behaviour", y="is_SW",
    capsize=0.05, width=0.8, order=behavioural_states,
    n_boot=10000, seed=42, ax=axs_behaviour, 
    color="gray", edgecolor="black"
)
axs_behaviour.set_ylim(0,0.2)
for i, x in enumerate(behavioural_states):
    axs_behaviour.text(
        str(x), 0.195, letters[i], ha="center", va="top", fontsize=10,
        backgroundcolor="white"
    )
axs_behaviour.set_xlabel("Behavioural state")
axs_behaviour.set_ylabel("SWs (in % of whistles)")
fig_behaviour.suptitle("Proportion of SWs among whistles per observed behavioural state.")
fig_behaviour.text(
    0.5, 0.91,
    "Different letter show significant differences according to Fisher's exact tests (p-values < 0.05).\nError bars show 95% confidence intervals.",
    ha="center", va="bottom", fontsize=8)
fig_behaviour.set_size_inches(5, 6)
fig_behaviour.subplots_adjust(
    top=0.9
)
axs_behaviour.xaxis.grid(True)
axs_behaviour.yaxis.grid(True)
fig_behaviour.savefig("/home/loic/Pictures/CDD/barplot_behaviours.pdf")
plt.show()
sns.reset_orig()

It looks like when dolphins are milling, they produce less SWs, in proportion, compared to the other behavioural states. Since, it is a resting behaviour, they are maybe communicating less between individuals, as resting individuals are less likely to respond anyway?

In [ ]:
# Specific SWT shapes per modality?
top_behaviour_fig, top_behaviour_axs, top_25_df = plot_wct_grid(
    results_df_several, name="SWT", n_categories=25, hue="behaviour",
    get_top_df=True)

top_behaviour_fig.subplots_adjust(
    bottom=0.075, top=0.875, 
    hspace=0.42, wspace=0.066)
top_behaviour_fig.suptitle("Top-25 Signature Whistles Types (SWTs) with the most elements.")
top_behaviour_fig.text(
    0.5, 0.925,
    "Each contour is represented with a different color depending on the behavioural state observed.",
    ha="center", fontsize=9)
top_behaviour_fig.set_size_inches(10, 7)
for axs in top_behaviour_axs:
    for ax in axs:
        ax.get_children()[-11].set_height(5)
        ax.set_title(
            ax.get_children()[-4].get_text(),
            pad=3, 
            fontsize=9,
            fontweight='bold')

top_behaviour_axs[-1, 0].text(
    0.5, 0.5, 'No observation', 
    ha='center', va='center', transform=top_behaviour_axs[-1, 0].transAxes)

top_behaviour_fig.savefig("/home/loic/Pictures/CDD/SWTs_25_behaviour.pdf")
plt.show()

In [ ]:
# Same grid but with histograms instead of contours
bar_top_behaviour_fig, bar_top_behaviour_axs = plt.subplots(
    5, 5, sharex=True, sharey=True, figsize=(10,7))
bar_top_behaviour_fig.subplots_adjust(
    left=0.066, right=0.95,
    bottom=0.15, top=0.91,
    wspace=0.1, hspace=0.5)

curr_grid = [0, 0]
for cat_id, cat_name in enumerate(top_25_df.category.unique()):
    bar_top_behaviour_axs[curr_grid[0],curr_grid[1]].set_title(
        f"SWT{cat_id+1:02d} (N={len(top_25_df[top_25_df.category == cat_name])})",
        pad=5, 
        fontsize=9,
        fontweight='bold')  
    
    sns.countplot(
        data=top_25_df[top_25_df.category == cat_name], 
        order=["Milling", "Socialising", "Travelling", "Foraging", "Attracted"],
        x="behaviour", width=0.5, color="gray", edgecolor="black",
        stat="proportion",
        ax=bar_top_behaviour_axs[curr_grid[0],curr_grid[1]]
    )
    bar_top_behaviour_axs[curr_grid[0],curr_grid[1]].set_xlabel("")
    bar_top_behaviour_axs[curr_grid[0],curr_grid[1]].set_ylabel("")

    if curr_grid[1] >= 5-1:
        curr_grid[0] += 1
        curr_grid[1] = 0
    else:
        curr_grid[1] += 1
  
for i in range(5):
    for j in range(5):
        bar_top_behaviour_axs[i, j].add_patch(
        plt.Rectangle(
            xy=(-0.5, 1.05), 
            width=5, 
            height=.3,
            facecolor='lightgray',
            clip_on=False,
            edgecolor="black",
            linewidth = .66))
        for label in bar_top_behaviour_axs[i, j].get_xticklabels():
            label.set_rotation(45)
            label.set_ha('right')

bar_top_behaviour_fig.suptitle("Distribution of Top-25 Signature Whistles Types (SWTs) per behaviour.")
bar_top_behaviour_fig.supylabel("Proportion of SWs")
bar_top_behaviour_fig.supxlabel("Behavioural state")
top_behaviour_fig.savefig("/home/loic/Pictures/CDD/barplot_SWTs_25_behaviour.pdf")
plt.show()

No obvious pattern: SWTs are emitted during all behavioural states. However, a SWTs is often emitted only during a specific observed behavioural state.

#### Beacon activation state

Does the DOLPHINFREE device has an influence on the emission of SWs?

In [ ]:
# test data
print("Fisher's exact tests:")
test_beacons = pairwise_tests(results_df, "clean_activation_state", "is_SW")
for test_beacon in test_beacons:
    print("\t", test_beacon[0], "VS", test_beacon[1], ":", f"{test_beacon[2]:.2e}", get_stars(test_beacon[2]))

In [ ]:
# Make plot
letters = ["a", "b", "a", "a", "a"] # manually from previous cell
activation_order = ["BEF", "BEF+DUR", "DUR", "DUR+AFT", "AFT"]

sns.set_style("ticks")
fig_beacon, axs_beacon = plt.subplots(1,1)
sns.barplot(
    data=results_df, x="clean_activation_state", y="is_SW",
    capsize=0.05, width=0.8, order=activation_order,
    n_boot=10000, seed=10, ax=axs_beacon,
    color="gray", edgecolor="black"
)
# since seaborn uses bootstrap, results are kinda different unfortunately.

axs_beacon.set_ylim(0,0.27)
for i, x in enumerate(activation_order):
    axs_beacon.text(
        str(x), 0.255, letters[i], ha="center", va="bottom", fontsize=11,
        backgroundcolor="white"
    )
axs_beacon.set_xlabel("Activation state of the beacon")
axs_beacon.set_ylabel("SWs (in % of whistles)")
axs_beacon.set_xticklabels(["Before", "Activation", "Activated", "Deactivation", "After"])
fig_beacon.suptitle("Proportion of SWs among whistles per beacon activation state.")
fig_beacon.text(
    0.5, 0.91,
    "Different letter show significant differences according to Fisher's exact tests (p-values < 0.05).\nError bars show 95% confidence intervals.",
    ha="center", va="bottom", fontsize=8)
fig_beacon.set_size_inches(6, 6)
fig_beacon.subplots_adjust(
    top=0.9
)
axs_beacon.xaxis.grid(True)
axs_beacon.yaxis.grid(True)
fig_beacon.savefig("/home/loic/Pictures/CDD/barplot_beacons.pdf")
plt.show()
sns.reset_orig()

It looks like, at the activation of the beacon, the ratio of SWs diminishes.
I do not really have an explanation for this, but it's interesting to note.

In [ ]:
# Specific SWT shapes per modality?
top_beacon_fig, top_beacon_axs, top_25_df = plot_wct_grid(
    results_df_several, name="SWT", n_categories=25, hue="clean_activation_state",
    get_top_df=True)

top_beacon_fig.subplots_adjust(
    bottom=0.075, top=0.875, 
    hspace=0.42, wspace=0.066)
top_beacon_fig.suptitle("Top-25 Signature Whistles Types (SWTs) with the most elements.")
top_beacon_fig.text(
    0.5, 0.925,
    "Each contour is represented with a different color depending on the activation state of the DOLPHINFREE beacon.",
    ha="center", fontsize=9)
top_beacon_fig.set_size_inches(10, 7)
for axs in top_beacon_axs:
    for ax in axs:
        ax.get_children()[-11].set_height(5)
        ax.set_title(
            ax.get_children()[-4].get_text(),
            pad=3, 
            fontsize=9,
            fontweight='bold')

top_beacon_fig.savefig("/home/loic/Pictures/CDD/SWTs_25_beacon.pdf")
plt.show()

In [ ]:
# Same grid but with histograms instead of contours
bar_top_beacon_fig, bar_top_beacon_axs = plt.subplots(
    5, 5, sharex=True, sharey=True, figsize=(10,7))
bar_top_beacon_fig.subplots_adjust(
    left=0.066, right=0.95,
    bottom=0.15, top=0.91,
    wspace=0.1, hspace=0.5)

curr_grid = [0, 0]
for cat_id, cat_name in enumerate(top_25_df.category.unique()):
    bar_top_beacon_axs[curr_grid[0],curr_grid[1]].set_title(
        f"SWT{cat_id+1:02d} (N={len(top_25_df[top_25_df.category == cat_name])})",
        pad=5, 
        fontsize=9,
        fontweight='bold')  
    
    sns.countplot(
        data=top_25_df[top_25_df.category == cat_name], 
        order=["BEF", "BEF+DUR", "DUR", "DUR+AFT", "AFT"],
        x="clean_activation_state", width=0.5, color="gray", edgecolor="black",
        stat="proportion",
        ax=bar_top_beacon_axs[curr_grid[0],curr_grid[1]]
    )
    bar_top_beacon_axs[curr_grid[0],curr_grid[1]].set_xlabel("")
    bar_top_beacon_axs[curr_grid[0],curr_grid[1]].set_ylabel("")

    if curr_grid[1] >= 5-1:
        curr_grid[0] += 1
        curr_grid[1] = 0
    else:
        curr_grid[1] += 1
  
for i in range(5):
    for j in range(5):
        bar_top_beacon_axs[i, j].add_patch(
        plt.Rectangle(
            xy=(-0.5, 1.05), 
            width=5, 
            height=.3,
            facecolor='lightgray',
            clip_on=False,
            edgecolor="black",
            linewidth = .66))
        for label in bar_top_beacon_axs[i, j].get_xticklabels():
            label.set_rotation(45)
            label.set_ha('right')

bar_top_beacon_fig.suptitle("Distribution of Top-25 Signature Whistles Types (SWTs) depending on the activation of the DOLPHINFREE beacon.")
bar_top_beacon_fig.supylabel("Proportion of SWs")
bar_top_beacon_fig.supxlabel("Activation state of the DOLPHINFREE beacon")
bar_top_beacon_fig.savefig("/home/loic/Pictures/CDD/barplot_SWTs_25_beacon.pdf")
plt.show()

Looks like more SWs are emitted before and during the activation for these top categories. Which is different than when taking them all into account. So nice visualisation, but biased.

#### Fishing net

An influence of the presence/absence of a fishing net?

In [ ]:
# test data
print("Fisher's exact tests:")
test_nets = pairwise_tests(results_df, "fishing_net", "is_SW")
for test_net in test_nets:
    print("\t", test_net[0], "VS", test_net[1], ":", f"{test_net[2]:.2e}", get_stars(test_net[2]))

In [ ]:
# Make plot
letters = ["a", "a"]

sns.set_style("ticks")
fig_net, axs_net = plt.subplots(1,1)
sns.barplot(
    data=results_df, x="fishing_net", y="is_SW",
    capsize=0.05, width=0.8,
    n_boot=10000, seed=10, ax=axs_net,
    color="gray", edgecolor="black"
)
axs_net.set_ylim(0,0.188)
for i, x in enumerate(["Absent", "Present"]):
    axs_net.text(
        str(x), 0.1781, letters[i], ha="center", va="bottom", fontsize=11,
        backgroundcolor="white"
    )

fig_net.suptitle("Proportion of SWs among whistles depending on the presence of a fishing net.")
fig_net.text(
    0.5, 0.91,
    "Different letter show significant differences according to Fisher's exact tests (p-values < 0.05).\nError bars show 95% confidence intervals.",
    ha="center", va="bottom", fontsize=8)
fig_net.set_size_inches(2, 6)
fig_net.subplots_adjust(
    top=0.9
)
axs_net.set_xlabel("Presence/Absence of fishing net")
axs_net.set_ylabel("SWs (in % of whistles)")
axs_net.xaxis.grid(True)
axs_net.yaxis.grid(True)
fig_net.savefig("/home/loic/Pictures/CDD/barplot_net.pdf")
plt.show()
sns.reset_orig()

Well, that one is easy to interpret at least. No influence of the fishing net.

#### Group size

The more dolphins, the more whistles, but are there also more SWs ?

In [ ]:
ratio_df = results_df.groupby(['sequence', 'group_size'])['is_SW'].mean().reset_index()

sns.set_style("ticks")
fig_size, axs_size = plt.subplots(1,1)
sns.regplot(
    data=ratio_df, x='group_size', y='is_SW',
    scatter=True, ax=axs_size,
    color="black"
)
fig_size.suptitle("Proportion of SWs among whistles depending on dolphin group sizes.")
# fig_size.set_size_inches(2, 6)
fig_size.subplots_adjust(
    top=0.92
)
axs_size.set_xlabel("Number of individuals per group.")
axs_size.set_ylabel("SWs (in % of whistles)")
axs_size.xaxis.grid(True)
axs_size.yaxis.grid(True)
fig_size.savefig("/home/loic/Pictures/CDD/regplot_group_size.pdf")
plt.show()
sns.reset_orig()

r = stats.spearmanr(
    ratio_df["group_size"], 
    ratio_df["is_SW"])

print(f"Spearman correlation: {r[0]:.2f} (p-value {r[1]:.2f})")

No relation between SW proportion and group size, we kinda expected it, but its still better to prove it.

#### Activation sequence VS behaviour

Let's try some chosen interactions, to investigate further.

In [ ]:
sns.set_style("ticks")
interaction_fig, interaction_axs = plt.subplots(1,1)
hue_order=["Milling", "Socialising", "Travelling", "Foraging", "Attraction"]
bars = sns.barplot(
    data=results_df, x="activation_state", y="is_SW", hue="behaviour",
    capsize=0.05, width=0.8, 
    order=["BEF", "BEF+DUR", "DUR", "DUR+AFT", "AFT"], 
    hue_order=hue_order,
    n_boot=10000, seed=42, ax=interaction_axs,
)

# Apply hatch patterns : ['/', '\\', '|', '-', '+', 'x', 'o', 'O', '.', '*']
unique_colors = np.unique(np.array([bar.get_facecolor() for bar in bars.patches]), axis=0)
hatch_patterns = ['.', '-', '/', '\\', 'o']
for bar in bars.patches:
    color = bar.get_facecolor()
    index = [i for i, unique_color in enumerate(unique_colors) if np.all(unique_color == color)]
    bar.set_hatch(hatch_patterns[index[0]])
    bar.set_facecolor('gray')
interaction_axs.legend(title="Behaviour", handlelength=3, handleheight=1.5)

interaction_fig.set_size_inches(10, 6)
interaction_fig.suptitle("Proportion of SWs among whistles depending on behavioural state and activation of the beacon.")
interaction_fig.subplots_adjust(
    top=0.92
)
interaction_axs.set_xlabel("Beacon activation state")
interaction_axs.set_ylabel("SWs (in % of whistles)")
interaction_axs.xaxis.grid(True)
interaction_axs.yaxis.grid(True)
interaction_axs.set_xticklabels(["Before", "Activation", "Activated", "Deactivation", "After"])
interaction_fig.savefig("/home/loic/Pictures/CDD/regplot_interaction_beahviour_beacon.pdf")
plt.show()
sns.reset_orig()

Too much variations, cannot determine true effects

#### Fishing net interactions

The presence of a fishing has no effect alone. But maybe when considering interactions it reveals an effect?

In [ ]:
sns.set_style("ticks")
net_interaction_fig, net_interaction_axs = plt.subplots(1,1)
bars = sns.barplot(
    data=results_df, x="activation_state", y="is_SW", hue="fishing_net",
    capsize=0.05, width=0.8, 
    order=["BEF", "BEF+DUR", "DUR", "DUR+AFT", "AFT"], 
    hue_order=["Absent", "Present"],
    n_boot=10000, seed=42, ax=net_interaction_axs,
)

net_interaction_fig.set_size_inches(6, 5)
net_interaction_fig.suptitle("Proportion of SWs among whistles depending on\nfishing net presence and activation of the beacon.")
net_interaction_fig.subplots_adjust(top=0.88)
net_interaction_axs.set_xlabel("Beacon activation state")
net_interaction_axs.set_ylabel("SWs (in % of whistles)")
net_interaction_axs.xaxis.grid(True)
net_interaction_axs.yaxis.grid(True)
plt.show()
sns.reset_orig()

In [ ]:
sns.set_style("ticks")
net_interaction_fig, net_interaction_axs = plt.subplots(1,1)
bars = sns.barplot(
    data=results_df, x="behaviour", y="is_SW", hue="fishing_net",
    capsize=0.05, width=0.8, 
    order=["Milling", "Socialising", "Travelling", "Foraging", "Attraction"], 
    hue_order=["Absent", "Present"],
    n_boot=10000, seed=42, ax=net_interaction_axs,
)

net_interaction_fig.set_size_inches(6, 5)
net_interaction_fig.suptitle("Proportion of SWs among whistles depending on\nfishing net presence and behavioural state.")
net_interaction_fig.subplots_adjust(top=0.88)
net_interaction_axs.set_xlabel("Observed behavioural state")
net_interaction_axs.set_ylabel("SWs (in % of whistles)")
net_interaction_axs.xaxis.grid(True)
net_interaction_axs.yaxis.grid(True)
plt.show()
sns.reset_orig()

## Conlusions - data exploration

Can we explain why 2021 has less SWs than the other years ? 
And why some days have significantly more SWs than others ?

We know that behaviour and beacon activation both have an effect on SWs. So let's test explore that path.

In [ ]:
print("Distribution of behavioural states per year:")
counts = results_df.groupby(['year', 'behaviour']).size().unstack(fill_value=0)
print(counts.div(counts.sum(axis=1), axis=0))

print("\nDistribution of activation states per year:")
counts = results_df.groupby(['year', 'clean_activation_state']).size().unstack(fill_value=0)
print(counts.div(counts.sum(axis=1), axis=0))

So in 2021, way more "BEF", but from previous "beacon activation state" analysis, this cannot explain it , not alone.

In [ ]:
variable_to_test = "behaviour"
local_df = results_df[(
    (results_df.date.dt.date.astype(str) == '2022-07-17') | 
    (results_df.date.dt.date.astype(str) == '2022-07-16') | 
    (results_df.date.dt.date.astype(str) == '2020-07-17') | 
    (results_df.date.dt.date.astype(str) == '2020-07-12')
)]

print("Distribution of behavioural states per day:")
counts = local_df.groupby(['date', 'behaviour']).size().unstack(fill_value=0)
print(counts.div(counts.sum(axis=1), axis=0))

print("\nDistribution of activation states per day:")
counts = local_df.groupby(['date', 'clean_activation_state']).size().unstack(fill_value=0)
print(counts.div(counts.sum(axis=1), axis=0))

From "Days" analysis, we expect 2022-07-17 and 2020-07-12 to have similar distributions. And 2020-07-17, 2022-07-17 similar distributions as well.

Well nothing is evident from these distributions.

### Discussion

There are probably more parameters, not estimated for this dataset, that can influence the number of SWs produced. We'll see this in the discussion.